# Corpus semantic & syntactic analysis
In this notebooks, I'll explore the following analyses of corpus semantics and syntactics:
1. **Vocabulary analysis.** Look for out-of-vocabulary words in each pairwise comparison of corpora. The more out-of-vocabulary words for a given pair, the less similar they are to one another. Expect that SciERC and PICKLE will have more overlapping vocabulary.
2. **Token analysis.** Using a word2vec model, embed the tokens from each corpus, and see how they cluster in the vector space. Expect that PICKLE and SciERC tokens will cluster together in space.
3. **Document analysis.** Using some kind of document embedding (doc2vec, a BERT based embedding, etc), perform the same style of analysis as for the tokens. Expect PICKLE and SciERC documents to cluster together more often.

## 1. Vocabulary analysis
### Getting the vocabulary from pre-trained models

In [1]:
from allennlp.models.model import Model
import dygie

In [2]:
scierc_model = Model.from_archive('https://s3-us-west-2.amazonaws.com/ai2-s2-research/dygiepp/master/scierc.tar.gz')

ConfigurationError: dygie not in acceptable choices for dataset_reader.type: ['babi', 'conll2003', 'interleaving', 'multitask', 'multitask_shim', 'sequence_tagging', 'sharded', 'text_classification_json']. You should either use the --include-package flag to make sure the correct module is loaded, or use a fully qualified class name in your config file like {"model": "my_module.models.MyModel"} to have it imported automatically.